In [5]:
!pip install pandas


In [6]:
import pandas as pd
import glob
import os

## Summary stats

In [64]:
# Define the file path
file_path = '/Users/aparnaj8/Box/InTrans/RWRAD_Internal/Final_files_with_variables/Final_list_of_variables/weekly_stats_demo_cleaned_for_model_v2.csv'

# Read the CSV file into a DataFrame
df_stat= pd.read_csv(file_path)

In [66]:
df_stat.head()

,subj,week_number,year,total_miles,total_trips,avg_miles_per_trip,total_minutes,avg_minutes_per_trip,non_nighttime_trips,percent_non_nighttime,...,mean_efficiency,mean_awakenings_mean_min,mean_awake_after_sleep_waso_min,errand_weekend_count,home_weekday_count,medical_weekday_count,none_weekday_count,social_weekday_count,Unnamed: 43,subj_type2
0,RWRAD_001,11,2021,81.858703,17,4.815218,284.533333,16.737255,17,100.0,...,NaN,NaN,NaN,1.0,6.0,1.0,NaN,NaN,NaN,healthy
1,RWRAD_001,12,2021,89.800539,22,4.081843,303.000000,13.772727,22,100.0,...,NaN,NaN,NaN,2.0,8.0,NaN,NaN,NaN,NaN,NaN
2,RWRAD_001,13,2021,311.730769,31,10.055831,612.866667,19.769892,31,100.0,...,0.865308,3.061806,45.250000,NaN,12.0,NaN,NaN,NaN,NaN,NaN
3,RWRAD_001,14,2021,83.971941,26,3.229690,311.983333,11.999359,26,100.0,...,0.879807,3.095035,45.714286,3.0,7.0,NaN,NaN,NaN,NaN,NaN
4,RWRAD_001,15,2021,151.992413,20,7.599621,296.400000,14.820000,20,100.0,...,0.875554,4.256156,53.857143,1.0,6.0,NaN,NaN,NaN,NaN,NaN


In [68]:
df_stat.dtypes

subj                                object
week_number                          int64
year                                 int64
total_miles                        float64
total_trips                          int64
avg_miles_per_trip                 float64
total_minutes                      float64
avg_minutes_per_trip               float64
non_nighttime_trips                  int64
percent_non_nighttime              float64
am_7_9_trips                         int64
percent_am_7_9                     float64
nighttime_80_percent                 int64
percent_nighttime_80               float64
pm_4_6_trips                         int64
percent_pm_4_6                     float64
within_15_miles                      int64
percent_within_15_miles            float64
within_25_miles                      int64
percent_within_25_miles            float64
trip_chains                          int64
avg_miles_per_chain                float64
avg_minutes_per_chain              float64
speed_60_mp

In [70]:
# Remove the row where 'subj' is equal to 'RWRAD_065'
df_stat_cleaned = df_stat[df_stat['subj'] != 'RWRAD_065']



In [72]:
df_stat_cleaned ['subj_type'].unique()

array(['mci', 'normalaging', 'alzheimers'], dtype=object)

In [74]:
df_stat_cleaned ['subj_type'] = df_stat_cleaned ['subj_type'].replace({'mci': 1, 'alzheimers': 1, 'normalaging': 0})


C:\Users\aparnaj8\AppData\Local\Temp\ipykernel_69316\4233975673.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stat_cleaned ['subj_type'] = df_stat_cleaned ['subj_type'].replace({'mci': 1, 'alzheimers': 1, 'normalaging': 0})


In [76]:
df_stat_cleaned.dtypes

subj                                object
week_number                          int64
year                                 int64
total_miles                        float64
total_trips                          int64
avg_miles_per_trip                 float64
total_minutes                      float64
avg_minutes_per_trip               float64
non_nighttime_trips                  int64
percent_non_nighttime              float64
am_7_9_trips                         int64
percent_am_7_9                     float64
nighttime_80_percent                 int64
percent_nighttime_80               float64
pm_4_6_trips                         int64
percent_pm_4_6                     float64
within_15_miles                      int64
percent_within_15_miles            float64
within_25_miles                      int64
percent_within_25_miles            float64
trip_chains                          int64
avg_miles_per_chain                float64
avg_minutes_per_chain              float64
speed_60_mp

## perform t test (not on demograhics here)

In [80]:
from scipy.stats import ttest_ind

# Select numerical columns
numerical_features = [
    'total_miles', 'total_trips', 'avg_miles_per_trip', 'total_minutes', 
    'avg_minutes_per_trip', 'non_nighttime_trips', 'percent_non_nighttime',
    'am_7_9_trips', 'percent_am_7_9', 'nighttime_80_percent', 
    'percent_nighttime_80', 'pm_4_6_trips', 'percent_pm_4_6', 'within_15_miles',
    'percent_within_15_miles', 'within_25_miles', 'percent_within_25_miles', 
    'trip_chains', 'avg_miles_per_chain', 'avg_minutes_per_chain', 
    'speed_60_mph', 'percent_speed_60_mph', 'avg_trip_speed', 'avg_trip_accel_x',
    'hard_brake_events', 'stoppage_probability', 'age', 
    'education_numberofyears_nacc', 'mean_efficiency', 'mean_awakenings_mean_min', 
    'mean_awake_after_sleep_waso_min', 'errand_weekend_count'
]

import pandas as pd
from scipy.stats import ttest_ind

# Set significance level
alpha = 0.05

# Store results
t_test_results = []

for feature in numerical_features:
    group_0 = df_stat_cleaned[df_stat_cleaned['subj_type'] == 0][feature].dropna()
    group_1 = df_stat_cleaned[df_stat_cleaned['subj_type'] == 1][feature].dropna()
    
    t_stat, p = ttest_ind(group_0, group_1, equal_var=False)  # Welch's t-test
    significance = "Significant" if p < alpha else "Not Significant"
    
    t_test_results.append([feature, t_stat, p, significance])

# Convert to DataFrame
t_test_df = pd.DataFrame(t_test_results, columns=["Feature", "T-Statistic", "P-Value", "Significance"])

# Display sorted by p-value
t_test_df.sort_values(by="P-Value", inplace=True)


In [82]:
t_test_df.head()

,Feature,T-Statistic,P-Value,Significance
29,mean_awakenings_mean_min,-7.818525,1.098413e-14,Significant
26,age,-4.820827,1.587972e-06,Significant
17,trip_chains,-4.738426,2.365775e-06,Significant
30,mean_awake_after_sleep_waso_min,-3.511334,4.607763e-04,Significant
2,avg_miles_per_trip,3.396413,7.138851e-04,Significant


In [61]:
# Save the final weekly statistics dataframe to a CSV file
t_test_df.to_csv('t_test_results1.csv', index=False)

In [38]:
# Calculate mean and standard deviation for all data
all_data_stats = df_stat_cleaned[numerical_features].agg(['mean', 'std'])

# Calculate mean and standard deviation for subj_type == 1
subj_type_1_stats = df_stat_cleaned[df_stat_cleaned['subj_type'] == 1][numerical_features].agg(['mean', 'std'])

# Calculate mean and standard deviation for subj_type == 0
subj_type_0_stats = df_stat_cleaned[df_stat_cleaned['subj_type'] == 0][numerical_features].agg(['mean', 'std'])


In [40]:
import pandas as pd

# Calculate mean and standard deviation for all data
all_data_stats = df_stat_cleaned[numerical_features].agg(['mean', 'std']).T

# Calculate mean and standard deviation for subj_type == 1
subj_type_1_stats = df_stat_cleaned[df_stat_cleaned['subj_type'] == 1][numerical_features].agg(['mean', 'std']).T

# Calculate mean and standard deviation for subj_type == 0
subj_type_0_stats = df_stat_cleaned[df_stat_cleaned['subj_type'] == 0][numerical_features].agg(['mean', 'std']).T

# Combine the results into "mean ± std" format
all_data_combined = (all_data_stats['mean'].round(2).astype(str) + ' ± ' + all_data_stats['std'].round(2).astype(str))
subj_type_1_combined = (subj_type_1_stats['mean'].round(2).astype(str) + ' ± ' + subj_type_1_stats['std'].round(2).astype(str))
subj_type_0_combined = (subj_type_0_stats['mean'].round(2).astype(str) + ' ± ' + subj_type_0_stats['std'].round(2).astype(str))

# Combine all results into a single DataFrame
combined_stats = pd.DataFrame({
    'Overall': all_data_combined,
    'subj_type == 1': subj_type_1_combined,
    'subj_type == 0': subj_type_0_combined
})

combined_stats.replace('Â±', '', inplace=True)


In [42]:
combined_stats.head(33)

,Overall,subj_type == 1,subj_type == 0
total_miles,129.3 ± 155.47,121.77 ± 139.53,140.79 ± 176.54
total_trips,21.8 ± 12.49,22.15 ± 13.07,21.25 ± 11.55
avg_miles_per_trip,5.68 ± 5.13,5.28 ± 3.87,6.29 ± 6.57
total_minutes,314.07 ± 275.72,320.75 ± 299.91,303.88 ± 233.95
avg_minutes_per_trip,14.72 ± 22.41,15.07 ± 28.19,14.18 ± 7.46
non_nighttime_trips,21.34 ± 12.07,21.65 ± 12.61,20.88 ± 11.2
percent_non_nighttime,98.23 ± 5.5,98.06 ± 5.73,98.48 ± 5.14
am_7_9_trips,3.51 ± 3.47,3.6 ± 3.63,3.37 ± 3.21
percent_am_7_9,16.04 ± 14.9,16.36 ± 15.51,15.56 ± 13.92
nighttime_80_percent,0.45 ± 1.25,0.51 ± 1.44,0.37 ± 0.88


In [66]:
# Save the final weekly statistics dataframe to a CSV file
combined_stats.to_csv('descriptive_stats_results.csv', index=True)

In [56]:
df_stat['gender'].unique()

array(['male', 'female'], dtype=object)

In [86]:
# Extract unique subjects from the 'subj' column
unique_subjects = df_stat_cleaned['subj'].drop_duplicates()

# Select the corresponding columns for 'gender', 'age', and 'education_numberofyears_nacc'
subject_info = df_stat_cleaned[['subj', 'gender', 'age', 'education_numberofyears_nacc', 'subj_type']].drop_duplicates(subset='subj')



In [88]:
subject_info.head()

,subj,gender,age,education_numberofyears_nacc,subj_type
0,RWRAD_001,male,79,18,1
14,RWRAD_002,male,82,14,0
27,RWRAD_003,male,85,19,0
39,RWRAD_004,female,88,13,0
53,RWRAD_005,female,70,14,0


In [62]:
len(subject_info)

118

In [50]:
# Count the number of females in all data
female_count_all = subject_info[subject_info['gender'].str.lower() == 'female'].shape[0]

# Count the total number of entries in all data
total_count_all = subject_info.shape[0]

# Count the number of females in subj_type == 1
female_count_subj_type_1 = subject_info[(subject_info['subj_type'] == 1) & (subject_info['gender'].str.lower() == 'female')].shape[0]

# Count the total number of entries in subj_type == 1
total_count_subj_type_1 = subject_info[subject_info['subj_type'] == 1].shape[0]

# Count the number of females in subj_type == 0
female_count_subj_type_0 = subject_info[(subject_info['subj_type'] == 0) & (subject_info['gender'].str.lower() == 'female')].shape[0]

# Count the total number of entries in subj_type == 0
total_count_subj_type_0 = subject_info[subject_info['subj_type'] == 0].shape[0]

# Calculate the percentage of females in all data
percentage_females_all = (female_count_all / total_count_all) * 100

# Calculate the percentage of females in subj_type == 1
percentage_females_subj_type_1 = (female_count_subj_type_1 / total_count_subj_type_1) * 100

# Calculate the percentage of females in subj_type == 0
percentage_females_subj_type_0 = (female_count_subj_type_0 / total_count_subj_type_0) * 100

# Print the results
print(f"Number of Females in All Data: {female_count_all} ({percentage_females_all:.2f}%)")
print(f"Number of Females in subj_type == 1: {female_count_subj_type_1} ({percentage_females_subj_type_1:.2f}%)")
print(f"Number of Females in subj_type == 0: {female_count_subj_type_0} ({percentage_females_subj_type_0:.2f}%)")


Number of Females in All Data: 61 (51.69%)
Number of Females in subj_type == 1: 29 (40.28%)
Number of Females in subj_type == 0: 32 (69.57%)


In [84]:
# Define the numerical features you want to calculate the stats for
numerical_features = ['age', 'education_numberofyears_nacc']  # You can add more numerical features here

# Calculate mean and standard deviation for all data
all_data_stats = subject_info[numerical_features].agg(['mean', 'std']).T

# Calculate mean and standard deviation for subj_type == 1
subj_type_1_stats = subject_info[subject_info['subj_type'] == 1][numerical_features].agg(['mean', 'std']).T

# Calculate mean and standard deviation for subj_type == 0
subj_type_0_stats = subject_info[subject_info['subj_type'] == 0][numerical_features].agg(['mean', 'std']).T

# Print the results
print("Statistics for all data:")
print(all_data_stats)
print("\nStatistics for subj_type == 1:")
print(subj_type_1_stats)
print("\nStatistics for subj_type == 0:")
print(subj_type_0_stats)


Statistics for all data:
                                   mean       std
age                           76.228814  6.037505
education_numberofyears_nacc  15.932203  2.544405

Statistics for subj_type == 1:
                                   mean       std
age                           76.875000  6.295398
education_numberofyears_nacc  15.888889  2.714362

Statistics for subj_type == 0:
                                   mean       std
age                           75.217391  5.525348
education_numberofyears_nacc  16.000000  2.280351


## t test on demographics

In [96]:
from scipy.stats import ttest_ind

# Select numerical columns
numerical_features = [ 'age', 
    'education_numberofyears_nacc'
]

import pandas as pd
from scipy.stats import ttest_ind

# Set significance level
alpha = 0.05

# Store results
t_test_results = []

for feature in numerical_features:
    group_0 = subject_info[subject_info['subj_type'] == 0][feature].dropna()
    group_1 = subject_info[subject_info['subj_type'] == 1][feature].dropna()
    
    t_stat, p = ttest_ind(group_0, group_1, equal_var=False)  # Welch's t-test
    significance = "Significant" if p < alpha else "Not Significant"
    
    t_test_results.append([feature, t_stat, p, significance])

# Convert to DataFrame
t_test_df = pd.DataFrame(t_test_results, columns=["Feature", "T-Statistic", "P-Value", "Significance"])

# Display sorted by p-value
t_test_df.sort_values(by="P-Value", inplace=True)


In [98]:
t_test_df.head()

,Feature,T-Statistic,P-Value,Significance
0,age,-1.504352,0.135496,Not Significant
1,education_numberofyears_nacc,0.239421,0.811235,Not Significant


## chi square on gender

In [90]:
from scipy.stats import chi2_contingency

# Select categorical columns
categorical_features = ['gender']

for feature in categorical_features:
    contingency_table = pd.crosstab(subject_info[feature], subject_info['subj_type'])
    chi2, p, dof, expected = chi2_contingency(contingency_table)
    print(f"Chi-Square Test for {feature}: p-value = {p:.4f}")


Chi-Square Test for gender: p-value = 0.0035
